# Interpolation

## Polynomial approximation

We'll write two programs to generate the interpolating polynomial as a vector in the space $\mathcal{P}_n$ of polynomials with degree at most $n$. The first will generate a basis of polynomials in the Lagrange form, the latter will do so in the Newton form.

### Lagrange form

Consider $f \in C[a,b]$ (the set of continuous real-valued functions on the compact interval $[a,b]$), and let $x_0,\ldots,x_n$ be $n+1$ distinct points. Does there exist polynomial of least degree $p \in \RR[x]$ which *interpolates* $f$ at the given points? such that $f(x_i) = p(x_i)$ for all $i \in \{0,1,\ldots,n\}$? Is it unique?

The fundamental theorem of algebra guarantees uniqueness, and the Lagrange form demonstrates existence.

Having $n+1$ distinct points $x_i$ specified, the $n+1$ Lagrange polynomials $\ell_k$ are written as the product $$\ell_k(x) = \prod_{i\neq k} \frac{x-x_i}{x_k-x_i} \text{ for all } k \in \{0,1,\ldots,n\}.$$

In [ ]:
def lagrange_basis(vecx):
    n = len(vecx)
    # choose n-1 of n entries
    hinges = [np.concatenate([vecx[:i], vecx[i+1:]]) for i in range(n)]
    # take product of n-1 terms, format into lambda functions
    polys = [lambda t, i=i: np.prod((t-hinges[i])/(vecx[i]-hinges[i]))\
            for i in range(n)]
    # returns a list of n functions, the Lagrange basis polynomials
    return polys


It can be shown that the set of $n+1$ Lagrange polynomials spans $\mathcal{P}_n$. We note the Lagrange polynomials form a basis for $\mathcal{P}_n$ since $\dim(\mathcal{P}_n) = n+1$.

For example

In [ ]:
vecx = np.array([0,1,2,3])

def lagrange_basis_plot(vecx):
    polys = lagrange_basis(vecx)
    start, stop = np.min(vecx), np.max(vecx)
    stiple = np.linspace(start,stop)
    # returns a list of plots of the basis polynomials, overlaid
    return [plt.plot(stiple,[polys[i](t) for t in stiple])\
            for i in range(len(vecx))]

lagrange_basis_plot(vecx)

Now, to show the existence of an interpolating polynomial, we'll specify a linear combination of basis vectors $\{\ell_k : k=0,1,\ldots,n\}$.

Let $f \in C[a,b]$, and specify $n+1$ distinct points $x_i$. The interpolating polynomial in Lagrange form is $p_n \in \mathcal{P}_n$ where $$p_n(x) = \sum_{k=0}^n f(x_k) \ell_k(x).$$

In [ ]:
def lagrange_coords(f,vecx):
    # returns the n coordinates of the interpolating polynomial
    # as a vector in $\mathcal{P}_n$ with the Lagrange basis
    return np.array([f(x) for x in vecx])

def interpol(coords,polys,t):
    # scales each vector by its coordinate
    scale = [lambda p, a=a: p * a for a in coords]
    # sums to form the desired linear combination
    # returns a (floating point) scalar
    return sum([scale[i](polys[i](t)) for i in range(len(vecx))])

Since the degree of a Lagrange polynomial generated by $n+1$ points is $n$, an evaluated at any of the $n+1$ generating points $x_i$, the Lagrange $k$th polynomial is equal to the Kroenecker delta $$\delta_{ik} = \begin{cases} 1 &\text{ if } i=k\\ 0 &\text{ else,} \end{cases}$$ we have the least degree interpolating polynomial that we sought the existence of, $p_n$. Moreover, $p_n$ is a unique vector in $\mathcal{P}_n$ (though it may have different representations, which can be more or less computationally efficient, by change of bases), so the existence proof for interpolating polynomials (generated by a finite number of distinct points) is finished.

Consider the function $f(x) = e^x$, and the points $\{0,1,2,3\}$. We have

In [ ]:
import math

f = lambda x: math.exp(x)

# we specify the interpolating poly, p
polys = lagrange_basis(vecx)
coords = lagrange_coords(f,vecx)
p = interpol(coords, polys, t)

# evaluating
print("exp(1.5) is approx  ", p(1.5))
print("exp(1.5) is exactly ", math.exp(1.5))
print("exp(4.0) is approx  ", p(4.0))
print("exp(4.0) is exactly ", math.exp(4.0))

# and graph p
start,stop = np.min(vecx),np.max(vecx)
stiple = np.linspace(start,stop)
fstiple = [interpol(coords, polys, t) for t in stiple]
plt.plot(stiple,fstiple)
plt.plot(stiple,np.array([f(t) for t in stiple]))

```python
def newton_basis(vecx):
    n = len(vecx)
    # returns a list of n functions, the newton form basis polynomials
    return [lambda t, j=j: np.prod([t-vecx[i] for i in range(j)])\
            for j in range(n)]

def newton_basis_plot(vecx):
    polys = newton_basis(vecx)
    start, stop = np.min(vecx), np.max(vecx)
    stiple = np.linspace(start,stop)
    # returns a list of plots of the basis polynomials, overlaid
    return [plt.plot(stiple,[polys[i](t) for t in stiple])\
            for i in range(len(vecx))]

def dd(f, vecx):
    if len(vecx)>1:
    # recursive call, returns the coordinate of the nth newton poly
        return (dd(f,vecx[1:]) - dd(f,vecx[:-1]))/(vecx[-1]-vecx[0])
    # floor of recursion
    else:
        return f(vecx[0])

def newton_coords(f, vecx):
    # TODO: save the recursive divided difference calls in memory
    # so we don't have to iterate to load them into this array
    return np.array([dd(f,vecx[:i+1]) for i in range(len(vecx))])


Let $f \in C^{n+1}[a,b]$, the $x_i$ be $n+1$ distinct points in $[a,b]$, and suppose $p_n$ is a polynomial of degree at most $n$ that interpolates $f$ at the $x_i$. Given $x \in [a,b]$, there exists a point $\xi \in (a,b)$ such that ... $$f(x) = p_n(x) + \frac{f^{(n+1)}(\xi)}{(n+1)!}\prod_{\forall i}(x-x_i).$$